In [61]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy

#import packages
import pandas as pd
import numpy as np
import datetime as dt

In [62]:
# Import data
data_crime = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/crime_data_edited.csv')
data_share = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/election_data_edited.csv')

In [63]:
# get into datetime format
data_crime['year'] = data_crime['date'].astype(str).str.split('.', expand=True)[2]
data_crime['month'] = data_crime['date'].astype(str).str.split('.', expand=True)[1]
data_crime['day'] = data_crime['date'].astype(str).str.split('.', expand=True)[0]
data_crime['date'] = pd.to_datetime(data_crime[['year', 'month', 'day']].astype(float), errors='ignore')

In [64]:
# Merge crime and candidate statistics with merge over city string
data_crime = data_crime[(data_crime['Land'] == 14) | (data_crime['Land'] == 8) | (data_crime['Land'] == 16)]
data_share['city'] = data_share['city'].str.replace(r'Stadt ', '')
data_share['city'] = data_share['city'].str.replace('Ä', 'A').str.replace('Ö', 'O').str.replace('Ü', 'U')

list_cities_share = data_share.sort_values('city')['city'].unique()
list_cities_crime = data_crime['city'].unique()

data_crime.rename({'Land':'state'}, axis=1, inplace=True)
data_crime['state'] = data_crime['state'].map({16:'TH', 8:'BW', 14:'SN'})

In [65]:
data_share.rename({'year':'cycle_1'}, axis=1, inplace=True)
data_share['cycle_2'] = data_share['cycle_1']
data_share['cycle_3'] = data_share['cycle_1']

Merging over years and cities? How many different cutoffs? Directly on election day, one month before and three months before

Cut-off days for the three states: 05.26.2019, 04.26.2019, 02.26.2019

In [66]:
# creating different cycle variables in crime data to merge over
data_crime['cycle_1'] = 2014
data_crime.loc[data_crime['date'] > dt.datetime(2019, 5, 26, 0, 0), 'cycle_1'] = 2019

data_crime['cycle_2'] = 2014
data_crime.loc[data_crime['date'] > dt.datetime(2019, 4, 26, 0, 0), 'cycle_2'] = 2019

data_crime['cycle_3'] = 2014
data_crime.loc[data_crime['date'] > dt.datetime(2019, 2, 26, 0, 0), 'cycle_3'] = 2019

In [35]:
pd.merge(data_crime, data_share, on=['cycle_1', 'city', 'party', 'state'], how='right', validate='m:1')

MergeError: Merge keys are not unique in right dataset; not a many-to-one merge

In [67]:
list_double = pd.DataFrame()
for city in data_share['city'].unique():
    
    data_city = data_share[data_share['city'] == city]
    for party in data_city['party'].unique():
        
        data_party = data_city[data_city['party'] == party]
        for election in data_party['cycle_1'].unique():
            
            data_election = data_party[data_party['cycle_1'] == election]
            
            if len(data_election) > 1:
                
                # append to dataset of 
                list_double = list_double.append(data_election)
                

In [68]:
len(list_double['city'].unique()), len(data_share['city'].unique())

(27, 1990)

In [60]:
list_double[list_double['state'] == 'BW']['city'].unique()

array(['Altdorf', 'Hochdorf', 'Durnau', 'Urbach', 'Talheim', 'Waldenburg',
       'Rosenberg', 'Westhausen', 'Malsch', 'Sulzfeld', 'Lichtenau',
       'Limbach', 'Walldorf', 'Schomberg', 'Seebach', 'Steinach',
       'Hohenstein', 'Altheim', 'Neukirch'], dtype=object)

In [69]:
list_double[list_double['city'] == 'Meissen']

,Unnamed: 0,city,party,Anzahl Bewerber,Prozent Frauen,darunter Frauen,cycle_1,state,cycle_2,cycle_3
13633,13633,Meissen,CDU,381.0,21.259843,81.0,2014,SN,2014,2014
13670,13670,Meissen,CDU,16.0,18.750000,3.0,2014,SN,2014,2014
14930,14930,Meissen,CDU,359.0,21.727019,78.0,2019,SN,2019,2019
14973,14973,Meissen,CDU,25.0,28.000000,7.0,2019,SN,2019,2019
13634,13634,Meissen,DIE LINKE,119.0,33.613445,40.0,2014,SN,2014,2014
13671,13671,Meissen,DIE LINKE,17.0,23.529412,4.0,2014,SN,2014,2014
14931,14931,Meissen,DIE LINKE,88.0,38.636364,34.0,2019,SN,2019,2019
14974,14974,Meissen,DIE LINKE,14.0,42.857143,6.0,2019,SN,2019,2019
13635,13635,Meissen,SPD,119.0,24.369748,29.0,2014,SN,2014,2014
13673,13673,Meissen,SPD,11.0,27.272727,3.0,2014,SN,2014,2014
